In [1]:
# load the dataset of CTL materials
import pickle

with open('df_all_ctl.pkl', 'rb') as f:
    df_all_ctl = pickle.load(f)

# Get Pubchem CID from name

In [25]:
import requests

def search_pubchem_by_name(compound_name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/cids/JSON"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
        return None

# Example usage: searching for a compound by name
compound_name = "spiro-meotad"
result = search_pubchem_by_name(compound_name)
CID = result['IdentifierList']['CID'][0]
print(CID)

16161850


# Get properties from CID

In [26]:
import requests
#r = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{CID}/property/CanonicalSMILES/JSON")
#print(r)

def get_properties_from_CID(CID):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{CID}/property/CanonicalSMILES/JSON"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        property = response.json()
        property = property['PropertyTable']['Properties'][0]['CanonicalSMILES']
        return property
    else:
        print(f"Error: Unable to fetch property. Status code: {response.status_code}")
        return None
    
result = get_properties_from_CID(CID)
print(result)


COC1=CC=C(C=C1)N(C2=CC=C(C=C2)OC)C3=CC4=C(C=C3)C5=C(C46C7=C(C=CC(=C7)N(C8=CC=C(C=C8)OC)C9=CC=C(C=C9)OC)C1=C6C=C(C=C1)N(C1=CC=C(C=C1)OC)C1=CC=C(C=C1)OC)C=C(C=C5)N(C1=CC=C(C=C1)OC)C1=CC=C(C=C1)OC


# Discarded code

In [7]:
# attempt fuzzy matching (does not work)

from fuzzywuzzy import process
import requests

def search_pubchem_top_matches(query, top_n):
    '''This function searches PubChem for the top N matches of a compound by name.
    '''
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{query}/cids/JSON"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        top_matches = data['IdentifierList']['CID'][:top_n]
        return top_matches
    else:
        print(f"Error in top matches retrieval. Status code: {response.status_code}")
        return None


def search_pubchem_by_name_fuzzy(query, top_n=10):
    '''This searches PubChem for a compound by name using fuzzy matching.
    '''
    # search for the top N matches to the name
    top_matches = search_pubchem_top_matches(query, top_n)
    
    if not top_matches:
        return None
    
    # Convert CIDs to compound names
    compound_names = [get_compound_name(cid) for cid in top_matches]
    
    # Use fuzzy matching to find the closest match to the provided compound name
    closest_match, score = process.extractOne(query, compound_names)
    
    # Search PubChem by the closest match
    result = search_pubchem_by_name(closest_match)
    return result


def get_compound_name(cid):
    '''This function retrieves the name of a compound from PubChem using its CID.
    '''
    # PubChem PUG REST API endpoint for compound information
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/Title/JSON"
    
    # Send GET request to the API
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON response
        data = response.json()
        # Extract compound name
        compound_name = data['PropertyTable']['Properties'][0]['Title']
        return compound_name
    else:
        # If request was unsuccessful, print error message
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
        return None

# Example usage: searching for a compound by name (with Top N matches and fuzzy matching)
compound_name = "aspirin"
result = search_pubchem_by_name_fuzzy(compound_name, top_n=5)
print(result)


Error in top matches retrieval. Status code: 404
None
